In [68]:
import pandas as pd
import altair as alt
from altair.expr import datum
import warnings
warnings.filterwarnings('ignore')

In [69]:
## Load the Dataset and the model Prediction
data_filename = '../data/Model/data.pkl'
df =  pd.read_pickle(data_filename)

In [70]:
dict_name_to_rank_df = {
    'shap': pd.read_pickle('../data/Attr/shap_rank.pkl'),
    'lime': pd.read_pickle('../data/Attr/lime_rank.pkl'),
    'ig': pd.read_pickle('../data/Attr/ig_rank.pkl'), 
    'deepLift': pd.read_pickle('../data/Attr/deepLift_rank.pkl')
}
dict_name_to_attr_df = {
    'shap':  pd.read_pickle('../data/Attr/Shap_nn.pkl'), 
    'lime':  pd.read_pickle('../data/Attr/Shap_nn.pkl'), 
    'ig': pd.read_pickle('../data/Attr/ig_attr.pkl'), 
    'deepLift': pd.read_pickle('../data/Attr/deepLift_attr.pkl'),
}

In [71]:
distance_signed_rank = pd.DataFrame()
i = 0
for method_name_i, rank_df_i in dict_name_to_rank_df.items():
    j = 0
    for method_name_j, rank_df_j in dict_name_to_rank_df.items():
        if i < j:
            distance_signed_rank[method_name_i + '_' + method_name_j] = ((rank_df_i- rank_df_j) ** 2).sum(axis=1) ** 0.5
        j +=1
    i +=1
    

In [72]:
data_and_distance_rank = pd.concat([df, distance_signed_rank], axis = 1)

In [73]:
brush = alt.selection(type="interval")
base = alt.Chart(data_and_distance_rank).encode(
    opacity = alt.condition(brush, alt.value(1.0), alt.value(0.2))
).add_selection(
    brush
).properties(
    width=350,
    height=350
)

In [74]:
def Get_distance_and_feature_value_plot(feature_name):
    view_distance = base.mark_bar().encode(
        x= alt.X('shap_lime', title = 'distance of signed feature rank between shap and lime attribution method', bin=alt.Bin(step = 1, extent=[0, 30])), 
        y='count()'
    )
    min_range = df.min()[feature_name]
    max_range = df.max()[feature_name]
    feature_value_bin_step = round((max_range - min_range)/30)
    
    view_feature_value = alt.Chart(data_and_distance_rank).mark_bar().encode(
        x= alt.X(feature_name, bin=alt.Bin(step = feature_value_bin_step, extent=[min_range, max_range])), 
        y = alt.Y('count()'),
        color = 'target:N'
    ).transform_filter(brush)
    
    return view_distance | view_feature_value

In [75]:
Get_distance_and_feature_value_plot('glucose')

alt.HConcatChart(...)